In [6]:
import numpy as np
from scipy.optimize import fsolve, root
import matplotlib.pyplot as plt

# 1. Parameters Definition
#    Based on GHKT (Golosov et al., 2014) and Barrage (2014) for the base model,
#    and Chazel et al. (2023) for mineral-specific parameters.

class Parameters:
    def __init__(self):
        # GHKT Baseline Parameters (from GHKT 2014 Table 1 and Barrage 2014 Table S-I)
        self.beta = 0.985**10     # Decadal discount factor (0.985 annual rate compounded over 10 years) 

        self.alpha = 0.3          # Output elasticity of capital 
        self.nu = 0.04            # Output elasticity of energy 
        self.gamma = 2.3793e-5    # Damage parameter (gamma_bar in GHKT) 
        self.phi_L = 0.2          # Share of carbon remaining permanently 
        self.phi_0 = 0.393        # Share of remaining emissions after immediate absorption 
        self.epsilon = (1 - 0.0228)**10 # Decadal decay factor for atmospheric carbon (annual 1-0.0228 compounded over 10 years) 

        self.rho_energy = -0.058  # Parameter for interfuel substitution 
        self.kappa1 = 0.5429      # Share parameter for oil in energy composite 
        self.kappa2 = 0.1015      # Share parameter for coal in energy composite 
        self.kappa3 = 1 - self.kappa1 - self.kappa2 # Share parameter for green energy (calculated as 1 - kappa1 - kappa2) 
        self.A2_0 = 7683          # Initial labor productivity in coal sector (A_2,0 in GHKT) 
        self.A3_0 = 1311          # Initial labor productivity in green sector (A_3,0 in GHKT) 
        self.omega2 = (1.02)**10 # Decadal growth rate for A2 (annual 1.02 compounded over 10 years) 
        self.omega3 = (1.02)**10 # Decadal growth rate for A3 (annual 1.02 compounded over 10 years) 
        self.S0_ghkt_oil_stock = 253.8 # Initial oil stock (GtC) in GHKT (R_0 in GHKT) 
        self.N_0 = 1              # Total labor supply (N_0 in GHKT) 
        self.Y0 = 700000          # Initial GDP per decade (Y_0 in GHKT) 

        # GHKT Atmospheric Carbon Initial Conditions (from GHKT 2014 Table 1)
        self.S_total_initial = 802      # Total atmospheric carbon at time 0 (year 2000 in GHKT) 
        self.S1_initial = 684           # Permanent component of atmospheric carbon at time 0 
        self.S2_initial = self.S_total_initial - self.S1_initial # Transient component, calculated 
        self.S_bar = 581                # Pre-industrial atmospheric CO2 concentration (GtC) (S_bar in GHKT) 

        # Chazel et al. (2023) Specific Parameters (Table 3 in Chazel et al.)
        self.kappa_s = 0.3085     # Share parameter for secondary minerals in green capital 
        self.kappa_p = 1 - self.kappa_s # Share parameter for primary minerals in green capital (calculated as 1-kappa_s) 
        self.rho_green_capital = 0.5 # Substitution parameter for primary/secondary minerals in green capital 
        self.Ap_0 = 132000        # Initial labor productivity in primary mineral extraction (A_p,0 in Chazel) 
        self.As_0 = 132000        # Initial labor productivity in secondary mineral extraction (A_s,0 in Chazel) 
        self.psi_green = 1.877    # Energy produced per unit of green capital (psi in Chazel) 
        self.kappa_G = 0.75       # Share parameter for green capital in green energy production 
        self.kappa_L = 1 - self.kappa_G # Share parameter for labor in green energy production (calculated as 1-kappa_G) 
        self.rho_green_energy = -3 # Substitution parameter for labor/green capital in green energy production (rho_bar in Chazel) 
        self.A3_0_chazel = 865.14 # Initial labor productivity in green energy sector (A_3,0 in Chazel) 
        self.gA3_chazel = (1.02)**10 # Decadal growth rate for A3_chazel (annual 1.02 compounded over 10 years) 
        self.gAs = (1.02)**10      # Decadal growth rate for As (annual 1.02 compounded over 10 years) 
        self.gAp = (1.02)**10      # Decadal growth rate for Ap (annual 1.02 compounded over 10 years) 
        self.Ms0 = 19             # Initial secondary mineral stock (MtCu) (M_s,0 in Chazel) 
        self.Mp0_scenario = 2000 # Scenario for initial primary mineral stock (MtCu). Chazel examines 50, 200, 1000, 2000. 
        self.T_horizon = 30       # Number of 10-year periods (300 years) as per Chazel et al. setup 

        # Consistency checks (optional, but good practice)
        if not np.isclose(self.kappa1 + self.kappa2 + self.kappa3, 1):
            print("Warning: Kappa values for energy do not sum to 1. Please check parameters.") [cite: 1, 3]
        if not np.isclose(self.kappa_s + self.kappa_p, 1):
            print("Warning: Kappa values for green capital do not sum to 1. Please check parameters.") [cite: 2]
        if not np.isclose(self.kappa_G + self.kappa_L, 1):
            print("Warning: Kappa values for green energy production do not sum to 1. Please check parameters.") [cite: 2]

# Instantiate parameters outside the class definition to use them
params = Parameters()

# Optional: Print to verify (can remove or comment out later)
print("Model Parameters Loaded:")
for attr, value in vars(params).items():
    print(f"  {attr}: {value}")

Model Parameters Loaded:
  beta: 0.859730442259143
  alpha: 0.3
  nu: 0.04
  gamma: 2.3793e-05
  phi_L: 0.2
  phi_0: 0.393
  epsilon: 0.7940257432864095
  rho_energy: -0.058
  kappa1: 0.5429
  kappa2: 0.1015
  kappa3: 0.3555999999999999
  A2_0: 7683
  A3_0: 1311
  omega2: 1.2189944199947573
  omega3: 1.2189944199947573
  S0_ghkt_oil_stock: 253.8
  N_0: 1
  Y0: 700000
  S_total_initial: 802
  S1_initial: 684
  S2_initial: 118
  S_bar: 581
  kappa_s: 0.3085
  kappa_p: 0.6915
  rho_green_capital: 0.5
  Ap_0: 132000
  As_0: 132000
  psi_green: 1.877
  kappa_G: 0.75
  kappa_L: 0.25
  rho_green_energy: -3
  A3_0_chazel: 865.14
  gA3_chazel: 1.2189944199947573
  gAs: 1.2189944199947573
  gAp: 1.2189944199947573
  Ms0: 19
  Mp0_scenario: 2000
  T_horizon: 30


In [5]:
# 2.1. Household Preferences

def utility_function(C_t, N_t):
    """
    Household utility function, logarithmic in per capita consumption.
    (Assumption 1 in GHKT 2014, and Eq 1 in Chazel et al. 2023 )
    C_t: Total consumption at time t
    N_t: Total labor supply/population at time t (N_t in GHKT and Chazel et al. )
    """
    # Ensure positive consumption and population to avoid log(0) or log(negative)
    if C_t <= 0 or N_t <= 0:
        return -np.inf # Represents very low utility for invalid consumption

    return np.log(C_t / N_t)

## QUESTIONS FOR MEETING

** Regarding utility_function definition
    * Gemini suggested the above code but I am confused on why they define utility as C_t, N_t as I thought in the papers they say utility is derived only from consumption
    

In [8]:
# 2.2. Core Production Functions (Retained by Chazel et al. from GHKT Baseline)

def energy_composite_decadal(E1_t_oil, E2_t_coal, E3_t_green_decadal, params):
    """
    Energy composite function E_t for decadal periods.
    (Eq 21 in GHKT 2014)
    E1_t_oil: Oil input at time t (E1,t in GHKT)
    E2_t_coal: Coal input at time t (E2,t in GHKT)
    E3_t_green_decadal: Green energy input at time t (E3,t in GHKT)
    params: Parameters object.
    """
    # E_t = (kappa1*E1,t^rho + kappa2*E2,t^rho + kappa3*E3,t^rho)^(1/rho) 
    # GHKT (2014) defines rho_energy = -0.058, which is a CES (Constant Elasticity of Substitution) form. 
    if params.rho_energy == 0:
        # This case is theoretically Cobb-Douglas (rho=0, product form), but not applicable with GHKT's specified rho_energy.
        return (params.kappa1 * E1_t_oil**params.rho_energy *
                params.kappa2 * E2_t_coal**params.rho_energy *
                params.kappa3 * E3_t_green_decadal**params.rho_energy)**(1/params.rho_energy)
    else:
        return (params.kappa1 * E1_t_oil**params.rho_energy +
                params.kappa2 * E2_t_coal**params.rho_energy +
                params.kappa3 * E3_t_green_decadal**params.rho_energy)**(1/params.rho_energy)

def final_goods_output_decadal(K_t, N0_t, E_t_composite_decadal, S_t_total, A0_t_initial, params, t_period):
    """
    Final goods production function Y_t for decadal periods.
    (Eq 20 in GHKT 2014) 
    K_t: Capital stock at time t
    N0_t: Labor allocated to final goods sector at time t
    E_t_composite_decadal: Composite energy input at time t
    S_t_total: Total atmospheric carbon stock at time t (S_t in GHKT notation) 
    A0_t_initial: Initial total factor productivity in final goods sector (exogenous, A0 from GHKT)
    params: Parameters object.
    t_period: The current decadal period (0, 1, 2, ..., T_horizon-1)
    """
    # Y_t = exp(-gamma * (S_t - S_bar)) * A0,t * K_t^alpha * N0,t^(1-alpha-nu) * E_t^nu 
    # GHKT assumes A0,t is constant in their benchmark for analytical solution. 
    # Chazel uses A0 as a fixed value in Table 2 for GHKT parameters. 
    
    A0_t = A0_t_initial # A0,t is constant in GHKT benchmark.

    damage_factor = np.exp(-params.gamma * (S_t_total - params.S_bar))
    return damage_factor * A0_t * (K_t**params.alpha) * \
           (N0_t**(1 - params.alpha - params.nu)) * (E_t_composite_decadal**params.nu)

# 2.3. Chazel et al. Specific Production Functions (Mineral & Green Capital)

def green_capital_stock(ms_t, mp_t, params):
    """
    Green capital production function G_t.
    (Eq 11 in Chazel et al. 2023) 
    ms_t: Flow of secondary (recycled) minerals (m_s,t in Chazel) 
    mp_t: Flow of primary minerals (m_p,t in Chazel) 
    params: Parameters object.
    """
    # G_t = (kappa_s*m_s,t^rho_green_capital + kappa_p*m_p,t^rho_green_capital)^(1/rho_green_capital) 
    # Chazel defines rho_green_capital = 0.5. 
    if params.rho_green_capital == 0: # Cobb-Douglas case (product form), not applicable with Chazel's rho.
        return (params.kappa_s * ms_t**params.rho_green_capital *
                params.kappa_p * mp_t**params.rho_green_capital)**(1/params.rho_green_capital)
    else:
        return (params.kappa_s * ms_t**params.rho_green_capital +
                params.kappa_p * mp_t**params.rho_green_capital)**(1/params.rho_green_capital)

def green_energy_production_chazel_decadal(A3_0_chazel_initial, gA3_chazel_growth_rate, t_period, N3_t, G_t, params):
    """
    Low-carbon energy production function E3_t.
    (Eq 10 in Chazel et al. 2023) 
    A3_0_chazel_initial: Initial labor productivity in green energy sector (A_3,0 in Chazel)
    gA3_chazel_growth_rate: Decadal growth rate for A3_chazel (g_A3 in Chazel)
    t_period: The current decadal period (0, 1, 2, ...)
    N3_t: Labor allocated to green energy production (N3,t in Chazel) 
    G_t: Green capital stock at time t (G_t in Chazel) 
    params: Parameters object.
    """
    A3_t_chazel = A3_0_chazel_initial * (gA3_chazel_growth_rate**t_period)
    
    # E3_t = [kappa_L*(A3,t*N3,t)^rho_green_energy + kappa_G*(psi_green*G_t)^rho_green_energy]^(1/rho_green_energy) 
    # Chazel defines rho_green_energy = -3. 
    if params.rho_green_energy == 0: # Cobb-Douglas case (product form), not applicable with Chazel's rho.
        # This will technically use params.rho_energy which is -0.058 from GHKT if rho_green_energy is 0.
        # However, since Chazel specifies rho_green_energy = -3, this block will not be hit.
        return (params.kappa_L * (A3_t_chazel * N3_t)**params.rho_green_energy *
                params.kappa_G * (params.psi_green * G_t)**params.rho_green_energy)**(1/params.rho_green_energy)
    else:
        # Corrected bug: changed **(1/params.rho_energy) to **(1/params.rho_green_energy)
        return (params.kappa_L * (A3_t_chazel * N3_t)**params.rho_green_energy +
                params.kappa_G * (params.psi_green * G_t)**params.rho_green_energy)**(1/params.rho_green_energy)

## TO DO: 
GO THROUGH ABOVE CODE FOR CONSISTENCY WITH PAPERS AND CHANGE REDUNDANT NAMING WHERE POSSIBLE
(the below part is just raw copy pasted without looking at it, to do later but start above)


In [9]:
# 3. State Variables and their Evolution

def update_capital_decadal(Y_t, C_t):
    """
    Capital accumulation for decadal periods.
    (Based on GHKT 2014 Eq 2 and Chazel et al. 2023 discussion of full depreciation over a decade)
    Y_t: Total output at time t
    C_t: Total consumption at time t
    """
    # GHKT (2014) and Chazel et al. (2023) assume full depreciation: K_t+1 = Y_t - C_t.
    investment = Y_t - C_t
    
    # Ensure non-negative capital. If investment is negative, capital can decrease.
    # The original model structure doesn't typically enforce K_t+1 >= 0 explicitly in this simple form,
    # but the solver would implicitly ensure feasible paths.
    return investment

def update_atmospheric_carbon_states_decadal(S1_prev, S2_prev, E_fossil_t, params):
    """
    Updates permanent (S1) and transient (S2) atmospheric carbon stocks for decadal periods.
    (Eq 16 in Chazel et al. 2023, derived from GHKT 2014 carbon cycle)
    S1_prev: Permanent carbon stock from previous period (S1,t-1)
    S2_prev: Transient carbon stock from previous period (S2,t-1)
    E_fossil_t: Total fossil fuel emissions at time t (Oil + Coal in carbon content units)
    params: Parameters object.
    """
    # S1,t = S1,t-1 + phi_L * E_fossil,t
    # (S2,t - S_bar) = epsilon_decadal * (S2,t-1 - S_bar) + phi_0_decadal * (1-phi_L) * E_fossil,t
    
    S1_next = S1_prev + params.phi_L * E_fossil_t
    S2_next = params.epsilon * (S2_prev - params.S_bar) + params.S_bar + params.phi_0 * (1 - params.phi_L) * E_fossil_t
    return S1_next, S2_next

def update_mineral_stocks_decadal(Mp_t_prev, ms_t, mp_t):
    """
    Updates primary (Mp) and secondary (Ms) mineral stocks for decadal periods.
    (Eq 14 and 17 in Chazel et al. 2023)
    Mp_t_prev: Primary mineral stock from previous period (R_i,t in GHKT for general resource, Mp_t in Chazel)
    ms_t: Flow of secondary (recycled) minerals at time t (m_s,t in Chazel)
    mp_t: Flow of primary minerals at time t (m_p,t in Chazel)
    """
    # Mp,t+1 = Mp,t - m_p,t
    # Ms,t+1 = Ms,t - m_s,t + (m_s,t + m_p,t) = Ms,t + m_p,t (simplifies to Ms_t+1 = current_ms_t + current_mp_t)
    
    Mp_next = Mp_t_prev - mp_t
    # In Chazel's Eq 17: M_s,t+1 = M_s,t + m_p,t. This implies that the entire primary mineral extracted in a period
    # immediately adds to the secondary stock available for recycling in the *next* period.
    # It also implies that all secondary minerals *extracted* in the current period (m_s,t) are consumed/used,
    # and their input to green capital production is then "depreciated" and contributes to the next period's secondary stock.
    # Chazel's wording in text (page 9) and equation (17) M_s,t+1 = M_s,t + m_p,t is subtle.
    # Let's stick to Chazel's explicit Eq 17, which means secondary stock increases by the primary mineral extracted in the current period.
    # This also implies that consumed secondary minerals *re-enter* the secondary stock in the next period.
    
    Ms_next = ms_t + mp_t # This is the interpretation of the output of the sector (m_s,t + m_p,t) becomes the next stock
    
    # Re-reading Chazel Eq 17: R_i,t+1 = R_i,t - E_i,t >=0 for general resources.
    # For secondary minerals M_s,t+1 = M_s,t - m_s,t + (m_s,t + m_p,t)
    # The (m_s,t + m_p,t) term represents the flow of new green capital that then depreciates into the secondary stock.
    # So the equation in the paper is: M_s,t+1 = M_s,t + m_p,t
    # This implies all of (m_s,t + m_p,t) from G_t production effectively becomes secondary stock, and m_s,t is consumed from M_s,t.
    
    Ms_next = ms_t + mp_t # Assuming this (m_s,t + m_p,t) is the total amount of green capital G_t
                             # which then depreciates into the secondary stock as per Chazel's text.
                             # If G_t = (kappa_s*m_s,t^rho + kappa_p*m_p,t^rho)^(1/rho)
                             # Then Ms_next = Ms_t_prev - ms_t + G_t. This is different.

    # Let's strictly follow Chazel's Eq (17) for secondary mineral stock evolution:
    # M_s,t+1 = M_s,t + m_p,t
    # This is a strong simplifying assumption in Chazel's paper, implying that all primary mineral extracted
    # permanently adds to the secondary stock available for recycling in all future periods,
    # and implicitly, that what is recycled (m_s,t) is immediately replaced by new primary material entering the stock.
    # So, the original formulation was correct.
    Ms_next = Ms_t_prev + mp_t # As per Chazel's explicit Eq (17) and text "Ms,t+1=Ms,t+mp,t".

    return Mp_next, Ms_next

In [17]:
# 4. First Order Conditions (FOCs) and System Equations for Solver

def full_system_equations(variables_flat, params):
    """
    Defines the full system of non-linear equations (FOCs and state transitions)
    to be solved simultaneously for all time steps.

    variables_flat: A 1D array of all variables (control and state) for all time steps.
    params: Parameters object.

    Returns:
        numpy.ndarray: A 1D array of residuals, where each element should be zero at the solution.
    """

    T = params.T_horizon # Total number of periods

    # --- 4.1. Define Variable Indices ---
    # This maps descriptive names to their positions within the flattened array for each period.
    # The order here must be strictly consistent with how variables_flat is initially constructed.

    # Control/Decision Variables (chosen by planner/market for each period t)
    IDX_C = 0       # Consumption
    IDX_N0 = 1      # Labor in final goods sector
    IDX_LC = 2      # Labor in coal sector
    IDX_N3 = 3      # Labor in green energy production (N3_t in Chazel)
    IDX_Np = 4      # Labor in primary mineral extraction
    IDX_Ns = 5      # Labor in secondary mineral extraction
    IDX_F = 6       # Oil extraction (E1,t in GHKT)
    IDX_mp = 7      # Primary mineral flow (m_p,t in Chazel)
    IDX_ms = 8      # Secondary mineral flow (m_s,t in Chazel)

    # State Variables (evolve over time, their values for t+1 are implicitly solved via transition equations)
    # These are treated as variables to solve for across time.
    IDX_Mu_oil = 9  # Oil scarcity rent
    IDX_Mu_p = 10   # Primary mineral scarcity rent
    IDX_Mu_s = 11   # Secondary mineral scarcity rent
    IDX_Lambda_s = 12 # Optimal carbon tax (marginal externality damage)
    IDX_S1 = 13     # Atmospheric Carbon Permanent Component (S1,t)
    IDX_S2 = 14     # Atmospheric Carbon Transient Component (S2,t)
    IDX_K = 15      # Capital Stock (K_t)
    IDX_G = 16      # Green Capital Stock (G_t)
    IDX_OilStock = 17 # Oil Stock (R_t in GHKT for oil)
    IDX_MpStock = 18 # Primary Mineral Stock (M_p,t in Chazel)
    IDX_MsStock = 19 # Secondary Mineral Stock (M_s,t in Chazel)

    NUM_VARS_PER_PERIOD = 20 # Corrected to 20 to match highest IDX + 1

    # --- 4.2. Initialize Arrays for Variables and Residuals ---
    # Reshape the flattened input array into a 2D array (T periods x NUM_VARS_PER_PERIOD)
    variables = variables_flat.reshape(T, NUM_VARS_PER_PERIOD)
    
    # Initialize list to store residuals (equations to be satisfied)
    residuals = []

    # --- 4.3. Handle Initial State Variables (at t=0) ---
    # These are fixed by calibration. We add residuals to ensure the solver uses these fixed starting values.
    # The `variables` array will have values for t=0 that need to match params.initial_values.

    # Residual for initial capital stock K[0]
    # GHKT provides analytical solution for K_0 = alpha * beta * Y_0 / (1 - beta * (1 - alpha))
    # Or matches K0 as given in Barrage S-II.
    # Barrage Table S-II K0 = 128920.
    residuals.append(variables[0, IDX_K] - 128920) # Match Barrage Table S-II K0 

    # Residual for initial atmospheric carbon S1[0]
    residuals.append(variables[0, IDX_S1] - params.S1_initial)

    # Residual for initial atmospheric carbon S2[0]
    residuals.append(variables[0, IDX_S2] - params.S2_initial)

    # Residual for initial oil stock (R_0 in GHKT)
    # The oil stock is in finite supply. 
    # The initial amount of oil R0 (GtC) is 253.8. 
    residuals.append(variables[0, IDX_OilStock] - params.S0_ghkt_oil_stock) 

    # Residual for initial primary mineral stock Mp[0]
    # The Chazel paper models four copper budget scenarios for wind energy production, ranging from Mp,0=50MtCu to Mp,0=2000MtCu. 
    # In this case, we use 2000 for params.Mp0_scenario.
    residuals.append(variables[0, IDX_MpStock] - params.Mp0_scenario)

    # Residual for initial secondary mineral stock Ms[0]
    # Chazel assumes that the initial secondary mineral stock is 19Mt copper. 
    residuals.append(variables[0, IDX_MsStock] - params.Ms0)
    
    # Residual for initial green capital G[0] (Chazel et al. does not explicitly state G0, often derived)
    # Assume 0 initial green capital for simplicity if not given.
    residuals.append(variables[0, IDX_G] - 0)

    # --- 4.4. Loop through time periods to define equations ---
    for t in range(T):
        # Extract variables for current period (t)
        C_t = variables[t, IDX_C]
        N0_t = variables[t, IDX_N0]
        LC_t = variables[t, IDX_LC]
        N3_t = variables[t, IDX_N3]
        Np_t = variables[t, IDX_Np]
        Ns_t = variables[t, IDX_Ns]
        F_t = variables[t, IDX_F]
        mp_t = variables[t, IDX_mp]
        ms_t = variables[t, IDX_ms]
        Mu_oil_t = variables[t, IDX_Mu_oil]
        Mu_p_t = variables[t, IDX_Mu_p]
        Mu_s_t = variables[t, IDX_Mu_s]
        Lambda_s_t = variables[t, IDX_Lambda_s]
        S1_t = variables[t, IDX_S1]
        S2_t = variables[t, IDX_S2]
        K_t = variables[t, IDX_K]
        G_t = variables[t, IDX_G]
        OilStock_t = variables[t, IDX_OilStock]
        MpStock_t = variables[t, IDX_MpStock]
        MsStock_t = variables[t, IDX_MsStock]

        # Extract variables for next period (t+1) - if t is not the last period
        # These are needed for FOCs (e.g., Hotelling rules that link t and t+1) and state updates.
        if t < T - 1:
            C_t_plus_1 = variables[t+1, IDX_C]
            Mu_oil_t_plus_1 = variables[t+1, IDX_Mu_oil]
            Mu_p_t_plus_1 = variables[t+1, IDX_Mu_p]
            Mu_s_t_plus_1 = variables[t+1, IDX_Mu_s]
            Lambda_s_t_plus_1 = variables[t+1, IDX_Lambda_s]
            S1_t_plus_1 = variables[t+1, IDX_S1]
            S2_t_plus_1 = variables[t+1, IDX_S2]
            K_t_plus_1 = variables[t+1, IDX_K]
            G_t_plus_1 = variables[t+1, IDX_G]
            OilStock_t_plus_1 = variables[t+1, IDX_OilStock]
            MpStock_t_plus_1 = variables[t+1, IDX_MpStock]
            MsStock_t_plus_1 = variables[t+1, IDX_MsStock]
            
            # Need next period's productivities for some FOCs (e.g. Hotelling)
            A2_t_plus_1 = params.A2_0 * (params.omega2_growth_rate**(t + 1))
            A3_t_plus_1 = params.A3_0 * (params.omega3_growth_rate**(t + 1))
            Ap_t_plus_1 = params.Ap_0 * (params.gAp_growth_rate**(t + 1))
            As_t_plus_1 = params.As_0 * (params.gAs_growth_rate**(t + 1))
            A3_t_chazel_plus_1 = params.A3_0_chazel * (params.gA3_chazel_growth_rate**(t + 1))


        # Derived intermediate variables for current period (t)
        # A0,t is 17887 as per Barrage Table S-II for calibration. 
        A0_t = 17887 
        A2_t = params.A2_0 * (params.omega2_growth_rate**t)
        A3_t = params.A3_0 * (params.omega3_growth_rate**t)
        Ap_t = params.Ap_0 * (params.gAp_growth_rate**t)
        As_t = params.As_0 * (params.gAs_growth_rate**t)
        A3_t_chazel = params.A3_0_chazel * (params.gA3_chazel_growth_rate**t)
        
        S_t_total = S1_t + S2_t # Total atmospheric carbon at current period
        N_t = params.N_0 # Total labor is constant in GHKT/Chazel baseline

        # Calculate energy composite and final output using current period values
        # E3_t is green energy production (from Chazel's modified function)
        E3_t = green_energy_production_chazel_decadal(A3_t_chazel, params.gA3_chazel_growth_rate, t, N3_t, G_t, params)
        
        # Total fossil emissions for carbon cycle update
        E_fossil_t = F_t + (A2_t * LC_t) # Oil extraction + Coal production

        # E_t_composite is total energy composite
        E_t_composite = energy_composite_decadal(F_t, (A2_t * LC_t), E3_t, params)
        
        # Y_t is final goods output
        Y_t = final_goods_output_decadal(K_t, N0_t, E_t_composite, S_t_total, A0_t, params, t)

        # --- Equations for each period t (residuals to be zero) ---

        # 1. Resource Constraint (Consumption + next Capital = Current Output)
        # GHKT Eq 2: C_t + K_t+1 = Y_t (simplified for full depreciation from GHKT Sec 3)
        # This equation defines K_t+1, so it should only be appended for t < T-1.
        if t < T - 1:
            residuals.append(C_t + K_t_plus_1 - Y_t) 

        # 2. Labor Constraint (Total Labor = Sum of Labor Allocations)
        # Chazel Eq 18: N0,t + N2,t + N3,t + Np,t + Ns,t = N_t
        # Here N2_t is LC_t, N3_t is N3_t, N_t is L_t.
        residuals.append(N0_t + LC_t + N3_t + Np_t + Ns_t - N_t)

        # 3. Capital Accumulation (K_t+1 update rule)
        # K_t+1 is implicitly defined by the resource constraint (Eq 1). We ensure the K_t+1 variable matches.
        # This is already handled by Eq 1 (C_t + K_t+1 - Y_t = 0).

        # 4. Atmospheric Carbon S1/S2 Accumulation (Chazel Eq 16)
        # S1_t+1, S2_t+1 are variables to be solved for.
        # We need to ensure they match the update rules.
        if t < T - 1:
            S1_next_derived, S2_next_derived = update_atmospheric_carbon_states_decadal(S1_t, S2_t, E_fossil_t, params)
            residuals.append(S1_t_plus_1 - S1_next_derived)
            residuals.append(S2_t_plus_1 - S2_next_derived)

        # 5. Oil Stock Update (GHKT Eq 1: R_i,t+1 = R_i,t - E_i,t)
        if t < T - 1:
            residuals.append(OilStock_t_plus_1 - (OilStock_t - F_t))

        # 6. Primary Mineral Stock Update (Chazel Eq 14: M_p,t+1 = M_p,t - m_p,t)
        if t < T - 1:
            residuals.append(MpStock_t_plus_1 - (MpStock_t - mp_t))

        # 7. Secondary Mineral Stock Update (Chazel Eq 17: M_s,t+1 = M_s,t + m_p,t)
        if t < T - 1:
            residuals.append(MsStock_t_plus_1 - (MsStock_t + mp_t))
        
        # 8. Green Capital Stock Definition (Chazel Eq 11: G_t = (kappa_s*m_s,t^rho + kappa_p*m_p,t^rho)^(1/rho))
        # G_t is directly produced from current flows of minerals.
        residuals.append(G_t - green_capital_stock(ms_t, mp_t, params))


        # --- Add FOCs (First Order Conditions) below ---
        # These are complex and will be added in subsequent parts.
        # They generally equate marginal benefits to marginal costs (including externality/scarcity rents).

    # --- 4.5. Terminal Conditions (for the last period T-1, specifically for variables at T) ---
    # These ensure consistency at the end of the simulation horizon.
    # Typically set scarcity rents of exhaustible resources to 0 if they are exhausted by the end.
    # Or enforce final stock values.

    # Total variables in variables_flat = T * NUM_VARS_PER_PERIOD.
    # The last period's variables (variables[T-1, ...])
    # The next period's variables for T-1 is variables[T, ...] which is out of bounds.
    # So, for any equation that refers to t+1 for the last period, we need to handle it outside the loop,
    # or ensure t+1 references are only for t < T-1.
    # We already have `if t < T - 1:` for transitions.
    # This leaves terminal FOCs (e.g. Hotelling for final period) and explicit terminal stock values.

    # Terminal Scarcity Rents (GHKT/Chazel assume oil exhausted, coal not)
    # Chazel paper implies oil is used up over the T_horizon, so its scarcity rent should be 0 at the end.
    # The problem has T periods from 0 to T-1. So period T is variables[T-1, ...] for its "next" value.
    # To set value for period T (index T) in the solution:
    # Need to add Mu_oil[T], Mu_p[T], Mu_s[T], S1[T], S2[T], K[T], G[T], OilStock[T], MpStock[T], MsStock[T] as variables.
    # This means NUM_VARS_PER_PERIOD is applied to T+1 steps, not T steps.
    # Let's adjust for T periods (0 to T-1), meaning state variables only go up to T-1 in the `variables` array.
    # The solver then needs to enforce values for S1[T], S2[T], K[T], OilStock[T], MpStock[T], MsStock[T].

    # The current setup defines variables from t=0 to t=T-1.
    # So variables[t+1] means variables[T] is out of scope.
    # We need to define terminal conditions on variables[T-1] for values in "next period" T.
    # A common way is to make Mu_oil[T-1] + Mu_oil[T] = 0 (approaching 0) or simply Mu_oil[T-1] = 0
    # Or to ensure the sum of extraction equals the initial stock.

    # Let's use the explicit terminal conditions from van der Ploeg et al. Mathematica setup for GHKT,
    # which sets the scarcity rents to 0 for the last period (T).
    # Since variables_flat only has T periods (0 to T-1), we will set the LAST period's scarcity rent to 0.
    # i.e., Mu_oil[T-1] = 0, Mu_p[T-1] = 0, Mu_s[T-1] = 0.

    # Terminal Scarcity Rent for Oil
    # The analytical solution for GHKT suggests oil is used up asymptotically and has a positive Hotelling rent,
    # not necessarily 0 at the finite horizon T. However, for numerical models, a common terminal condition is 0.
    # Chazel paper's quantitative implementation (page 5) mentions finite horizon T.
    residuals.append(Mu_oil_t_plus_1 - 0) # This should apply at t=T-1 to Mu_oil[T-1] and next period index is T.
                                          # This requires special indexing for t=T-1

    # To simplify terminal conditions for the solver for now:
    # We will assume that all scarcity rents (Mu_oil, Mu_p, Mu_s) are zero at the very last period (T-1).
    # This reduces the number of equations to specify here.
    # The Hotelling rule equations (linking Mu_t and Mu_t+1) will be added later.

    return np.array(residuals) # Return the flattened array of residuals